In [143]:
import os
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression as sk_OLS
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import math

from sklearn.metrics import r2_score
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression

In [144]:
def scaller(edu):
    x_max = max(edu)
    x_min = min(edu)
    scaled_xs =[]
    for x in edu:
        x_scaled = 4*(x-x_min)/(x_max-x_min)+1
        scaled_xs.append(x_scaled)
    return scaled_xs

In [145]:
longitudinal = pd.read_csv("datacsv/oasis_longitudinal.csv")
cross_sectional = pd.read_csv("datacsv/oasis_cross-sectional.csv")

edu = longitudinal.EDUC
scalled_edu = scaller(edu)
longitudinal["Educ"] = scalled_edu

frames = [cross_sectional, longitudinal]


combined = pd.concat(frames)
# combined = combined.rename(columns={"Group": "CDR"})
# combined = combined.map(rename)
# combined["Group"] = combined["Group"].astype('float')
# print(combined)
combined.reset_index(inplace=True, drop=True)


print(combined)
combined.to_csv("combined.csv")

                ID M/F Hand  Age      Educ  SES  MMSE  CDR  eTIV   nWBV  \
0    OAS1_0001_MR1   F    R   74  2.000000  3.0  29.0  0.0  1344  0.743   
1    OAS1_0002_MR1   F    R   55  4.000000  1.0  29.0  0.0  1147  0.810   
2    OAS1_0003_MR1   F    R   73  4.000000  3.0  27.0  0.5  1454  0.708   
3    OAS1_0004_MR1   M    R   28       NaN  NaN   NaN  NaN  1588  0.803   
4    OAS1_0005_MR1   M    R   18       NaN  NaN   NaN  NaN  1737  0.848   
..             ...  ..  ...  ...       ...  ...   ...  ...   ...    ...   
804            NaN   M    R   82  3.352941  1.0  28.0  0.5  1693  0.694   
805            NaN   M    R   86  3.352941  1.0  26.0  0.5  1688  0.675   
806            NaN   F    R   61  2.647059  2.0  30.0  0.0  1319  0.801   
807            NaN   F    R   63  2.647059  2.0  30.0  0.0  1327  0.796   
808            NaN   F    R   65  2.647059  2.0  30.0  0.0  1333  0.801   

       ASF  Delay Subject ID         MRI ID        Group  Visit  MR Delay  \
0    1.306    NaN     

In [146]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

cdr_na = combined["CDR"].isna()
# print(combined[cdr_na])


unlabled = combined[cdr_na]
labled = combined[~cdr_na]


print(len(labled)/len(combined))

y_label = labled.CDR
X_label = labled.drop([ "M/F", "Hand", "Subject ID", "MRI ID", "ID", "Group", "Visit", "EDUC", "Delay", "ASF"], axis=1)
# X_cluster_label = X_label.drop(["MR Delay", "SES", "MMSE"], axis=1)

# , "SES", "MMSE", "ASF"
x_unlabeled = unlabled.drop([ "M/F", "Hand", "Subject ID", "MRI ID", "ID", "Group", "Visit", "EDUC", "Delay", "ASF"], axis=1)
print(X_label)


0.7515451174289246
     Age      Educ  SES  MMSE  CDR  eTIV   nWBV  MR Delay
0     74  2.000000  3.0  29.0  0.0  1344  0.743       NaN
1     55  4.000000  1.0  29.0  0.0  1147  0.810       NaN
2     73  4.000000  3.0  27.0  0.5  1454  0.708       NaN
8     74  5.000000  2.0  30.0  0.0  1636  0.689       NaN
9     52  3.000000  2.0  30.0  0.0  1321  0.827       NaN
..   ...       ...  ...   ...  ...   ...    ...       ...
804   82  3.352941  1.0  28.0  0.5  1693  0.694     842.0
805   86  3.352941  1.0  26.0  0.5  1688  0.675    2297.0
806   61  2.647059  2.0  30.0  0.0  1319  0.801       0.0
807   63  2.647059  2.0  30.0  0.0  1327  0.796     763.0
808   65  2.647059  2.0  30.0  0.0  1333  0.801    1608.0

[608 rows x 8 columns]


In [147]:
# normalize values:
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer


imp_mean = IterativeImputer(random_state=42)
X_label = imp_mean.fit_transform(X_label)
X_label = pd.DataFrame(X_label, columns=["Age","Educ",  "SES",  "MMSE",  "CDR",  "eTIV",   "nWBV",  "MR Delay"])
X = x_unlabeled
x_infered_labels = imp_mean.transform(X)
x_infered_labels = pd.DataFrame(x_infered_labels, columns=["Age", "Educ", "SES", "MMSE", "CDR", "eTIV", "nWBV", "MR Delay"])
frames = [X_label, x_infered_labels]


combined = pd.concat(frames)
y_label = combined.CDR
combined = combined.drop(["CDR"], axis=1)


In [148]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=6)
x_components = pca.fit_transform(combined)
# print(pca.explained_variance_ratio_)
# print(pca.singular_values_)

In [149]:
X_train, X_test, y_train, y_test = train_test_split(x_components, y_label)
print(len(X_train))
# print(y_train)
print(len(X_test))
print(X_train)
# print(X_test)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

606
203
[[-7.05442724e+01 -1.63423433e+02 -1.02458329e+01 -3.00786002e+00
   2.38239280e+00 -4.83707951e-01]
 [ 5.89419777e+02 -1.81881156e+02 -1.06065910e+01 -1.46452817e+00
  -7.26418791e-01  2.40617318e-01]
 [ 9.83419190e+00  3.90419791e+01  4.08255203e+01 -3.87601453e-02
  -2.75005721e-02  6.22068637e-03]
 ...
 [-6.13041644e+02 -2.92400061e+02 -8.32304684e+00  2.32812894e+00
  -2.17349246e+00 -3.04193077e-01]
 [ 3.52112198e+01  9.23306928e+01  3.13355066e+01  3.57249318e-01
  -1.75820986e-02  6.10786912e-03]
 [-5.91572272e+02  1.88403438e+01 -1.23241174e+01 -2.11274729e+00
   7.03929349e-01 -2.47442924e-01]]


In [150]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf_svm.fit(X_train, y_train)
y_pred = clf_svm.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))




# y_unlabeled = clf_svm.predict(x_unlabeled)
# print(y_unlabeled)

Accuracy:  0.9113300492610837


In [151]:
x_un_components = pca.fit_transform(x_infered_labels )

y_unlabeled = clf_svm.predict(x_un_components)
print(y_unlabeled)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
